<a href="https://colab.research.google.com/github/NoahSeo74/Leanrning-project/blob/master/Masterthisis_2(bert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 9.6MB/s 
     |████████████████████████████████| 901kB 54.2MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 


In [2]:
import re
import random
import time
import datetime

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from scipy import stats
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split

In [3]:
def separator(text):
    return re.sub(pattern="<b.*/>|\.|\?|\!", string=text, repl=" [SEP] ")

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
df = pd.read_csv('/content/gdrive/My Drive/imdbdata.csv', header=0)
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
texts = df.review
classes = df.sentiment
classes_oh = df.sentiment.apply(lambda x: 1 if x == 'positive' else 0)
classes_oh = classes_oh.values

In [7]:
'''def BertTokenize(data):
   sentences =["[CLS] "+ str(sent) +" [SEP]" for sent in data ]
   tokenized_texts= [tokenizer.tokenize(sent) for sent in sentences]
   result =[]
   
   for text in tokenized_texts:
      encoded_sent = tokenizer.convert_tokens_to_ids(text)
      result.append(encoded_sent)
       
   return result
'''
def BertTokenize(data,tokenizer):
       result =[]
       for sent in data:
           encoded_sent = tokenizer.encode(sent,add_special_tokens =True,max_length=128)
           result.append(encoded_sent)
       return result
def pad_sentences(data,max_length=128):
    data = pad_sequences(data,maxlen=max_length,dtype='long',truncating='post',padding='post')


In [8]:
def create_masks(sents):
  
    attention_masks = []
    for seq in sents:
     seq_mask = [float(i>0) for i in seq]
     attention_masks.append(seq_mask)
  
    return attention_masks

def convert_data(sent):
   tokenized_sent = BertTokenize(list(sent),tokenizer)
   padded_sent = pad_sentences(tokenized_sent)

   masks_sent = create_masks(padded_sent)

   inputs = torch.tensor(padded_sent)
   masks = torch.tensor(masks_sent)


   return inputs, masks

def test_sentences(sent):
     model.eval()
     inputs, masks =convert_data(sent)
     b_input_ids = inputs.to(device)
     b_input_mask = masks.to(device)
    
    # no compute gradient
     with torch.no_grad():     
        # Forward 
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
   

     logits = outputs[0]
     pred = np.argmax(logits)
     if pred == 1:
        result = "positive"
     elif pred == 0: 
        result = "negative"

     return result    

In [9]:
def bert_tokenize(texts):
    return [tokenizer.tokenize("[CLS] " + text + " [SEP]") for text in texts]

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized = bert_tokenize(texts)
bert_ids = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized]

In [11]:
number_of_tokens = np.array([len(bert_id) for bert_id in bert_ids])
stats.describe(number_of_tokens)

DescribeResult(nobs=50000, minmax=(10, 3157), mean=310.32116, variance=53426.660389462195, skewness=2.165013432807556, kurtosis=6.800663656410416)

In [12]:
MAX_LEN = 512
padded_bert_ids = pad_sequences(bert_ids, maxlen=MAX_LEN, dtype='long',
									truncating='post', padding='post')
padded_bert_ids[0]

array([  101,  2028,  1997,  1996,  2060, 15814,  2038,  3855,  2008,
        2044,  3666,  2074,  1015, 11472,  2792,  2017,  1005,  2222,
        2022, 13322,  1012,  2027,  2024,  2157,  1010,  2004,  2023,
        2003,  3599,  2054,  3047,  2007,  2033,  1012,  1026,  7987,
        1013,  1028,  1026,  7987,  1013,  1028,  1996,  2034,  2518,
        2008,  4930,  2033,  2055, 11472,  2001,  2049, 24083,  1998,
        4895, 10258,  2378,  8450,  5019,  1997,  4808,  1010,  2029,
        2275,  1999,  2157,  2013,  1996,  2773,  2175,  1012,  3404,
        2033,  1010,  2023,  2003,  2025,  1037,  2265,  2005,  1996,
        8143, 18627,  2030,  5199,  3593,  1012,  2023,  2265,  8005,
        2053, 17957,  2007, 12362,  2000,  5850,  1010,  3348,  2030,
        4808,  1012,  2049,  2003, 13076,  1010,  1999,  1996,  4438,
        2224,  1997,  1996,  2773,  1012,  1026,  7987,  1013,  1028,
        1026,  7987,  1013,  1028,  2009,  2003,  2170, 11472,  2004,
        2008,  2003,

In [13]:
attention_masks = []
for seq in padded_bert_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [14]:
X_train, X_test, y_train, y_test = \
train_test_split(padded_bert_ids, classes_oh, random_state=42, test_size=0.3)

masks_train, masks_test, _, _ = train_test_split(attention_masks, padded_bert_ids, 
                                                 random_state=42, test_size=0.3)

X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, random_state=42, test_size=0.1)

masks_train, masks_val, _, _ = train_test_split(masks_train, masks_train, 
                                                 random_state=42, test_size=0.1)
display(
    f"X_train: {X_train.shape}",
    f"X_val: {X_val.shape}",
    f"X_test: {X_test.shape}",
    f"y_train: {y_train.shape}",
    f"y_val: {y_val.shape}",
    f"y_test: {y_test.shape}",
    f"masks_train: {len(masks_train)}",
    f"masks_val: {len(masks_val)}",
    f"masks_test: {len(masks_test)}",
)

'X_train: (31500, 512)'

'X_val: (3500, 512)'

'X_test: (15000, 512)'

'y_train: (31500,)'

'y_val: (3500,)'

'y_test: (15000,)'

'masks_train: 31500'

'masks_val: 3500'

'masks_test: 15000'

In [15]:
train_inputs = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
train_masks = torch.tensor(masks_train)
validation_inputs = torch.tensor(X_val)
validation_labels = torch.tensor(y_val)
validation_masks = torch.tensor(masks_val)

test_inputs = torch.tensor(X_test)
test_labels = torch.tensor(y_test)
test_masks = torch.tensor(masks_test)

print(train_inputs.shape)
print(train_labels.shape)
print(train_masks.shape)
print(validation_inputs.shape)
print(validation_labels.shape)
print(validation_masks.shape)
print(test_inputs.shape)
print(test_labels.shape)
print(test_masks.shape)

torch.Size([31500, 512])
torch.Size([31500])
torch.Size([31500, 512])
torch.Size([3500, 512])
torch.Size([3500])
torch.Size([3500, 512])
torch.Size([15000, 512])
torch.Size([15000])
torch.Size([15000, 512])


In [16]:
BATCH_SIZE = 4

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [17]:
device = torch.device("cuda:0")

In [18]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

epochs = 2
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# display time fuction 
def format_time(elapsed):

    # round
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss format change
    return str(datetime.timedelta(seconds=elapsed_rounded))
    

# fix random seed
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# initialize gradient
model.zero_grad()

# repeat epoch
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # set initial time
    t0 = time.time()

    # initialize loss
    total_loss = 0

    # change to train_mode
    model.train()
        
    # repeat batch and bring it from dataloader
    for step, batch in enumerate(train_dataloader):
        # display process information 
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # batch into GPU
        batch = tuple(t.to(device) for t in batch)
        
        # extract data in batch
        b_input_ids, b_input_mask, b_labels = batch

        # Forward                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # loss
        loss = outputs[0]

        # compute total loss
        total_loss += loss.item()

        # compute gradient using backward
        loss.backward()

        # clip gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # weigh parameter update through gradient
        optimizer.step()

        # lower learning rate using scheuler
        scheduler.step()

        # initalize gradient
        model.zero_grad()

    # compute average loss 
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    # set initial time
    t0 = time.time()

    # to change evaluation mode
    model.eval()

    # initalize variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # repeat batch and bring it from dataloader
    for batch in validation_dataloader:
        # batch into GPU
        batch = tuple(t.to(device) for t in batch)
        
        # extract data in batch
        b_input_ids, b_input_mask, b_labels = batch
        
        # no compute gradient
        with torch.no_grad():     
            # Forward 
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # loss
        logits = outputs[0]

        # all data to cpu
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # comparing logits with label, and compute accuracy
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")

print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch   500  of  7,875.    Elapsed: 0:01:14.
  Batch 1,000  of  7,875.    Elapsed: 0:02:27.
  Batch 1,500  of  7,875.    Elapsed: 0:03:41.
  Batch 2,000  of  7,875.    Elapsed: 0:04:54.
  Batch 2,500  of  7,875.    Elapsed: 0:06:08.
  Batch 3,000  of  7,875.    Elapsed: 0:07:21.
  Batch 3,500  of  7,875.    Elapsed: 0:08:34.
  Batch 4,000  of  7,875.    Elapsed: 0:09:48.
  Batch 4,500  of  7,875.    Elapsed: 0:11:02.
  Batch 5,000  of  7,875.    Elapsed: 0:12:15.
  Batch 5,500  of  7,875.    Elapsed: 0:13:29.
  Batch 6,000  of  7,875.    Elapsed: 0:14:42.
  Batch 6,500  of  7,875.    Elapsed: 0:15:56.
  Batch 7,000  of  7,875.    Elapsed: 0:17:09.
  Batch 7,500  of  7,875.    Elapsed: 0:18:22.

  Average training loss: 0.36
  Training epcoh took: 0:19:17

Running Validation...
  Accuracy: 0.94
  Validation took: 0:00:37

======== Epoch 2 / 2 ========
Training...
  Batch   500  of  7,875.    Elapsed: 0:01:13.
  Batch 1,000  of  7,875.    Elap

In [21]:
#from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# set inital time
t0 = time.time()


# to change evalution mode
model.eval()

# initialize varibles
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# repeat batch and bring it from dataloader
for step, batch in enumerate(test_dataloader):
    # display process information 
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # batch into GPU
    batch = tuple(t.to(device) for t in batch)
    
    # extract data in batch
    b_input_ids, b_input_mask, b_labels = batch
    
    # no compute gradient
    with torch.no_grad():     
        # Forward 
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # loss
    logits = outputs[0]

    # change all data to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # comparing logits with label, and compute accuracy
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
#print("f1score: {0:.2f}".format(eval_f1score/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  3,750.    Elapsed: 0:00:04.
  Batch   200  of  3,750.    Elapsed: 0:00:09.
  Batch   300  of  3,750.    Elapsed: 0:00:13.
  Batch   400  of  3,750.    Elapsed: 0:00:17.
  Batch   500  of  3,750.    Elapsed: 0:00:21.
  Batch   600  of  3,750.    Elapsed: 0:00:26.
  Batch   700  of  3,750.    Elapsed: 0:00:30.
  Batch   800  of  3,750.    Elapsed: 0:00:34.
  Batch   900  of  3,750.    Elapsed: 0:00:39.
  Batch 1,000  of  3,750.    Elapsed: 0:00:43.
  Batch 1,100  of  3,750.    Elapsed: 0:00:47.
  Batch 1,200  of  3,750.    Elapsed: 0:00:51.
  Batch 1,300  of  3,750.    Elapsed: 0:00:56.
  Batch 1,400  of  3,750.    Elapsed: 0:01:00.
  Batch 1,500  of  3,750.    Elapsed: 0:01:04.
  Batch 1,600  of  3,750.    Elapsed: 0:01:09.
  Batch 1,700  of  3,750.    Elapsed: 0:01:13.
  Batch 1,800  of  3,750.    Elapsed: 0:01:17.
  Batch 1,900  of  3,750.    Elapsed: 0:01:21.
  Batch 2,000  of  3,750.    Elapsed: 0:01:26.
  Batch 2,100  of  3,750.    Elapsed: 0:01:30.
  Batch 2,200

In [22]:
def convert_input_data(sentences):
  tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]
  Max_len = 128
  input_ids = [tokenizer.convert_tokens_to_ids(sent) for sent in tokenized_text]
  input_ids = pad_sequences(input_ids, maxlen = Max_len, padding = 'post', truncating='post',dtype = 'long')
  attention_mask = []
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_mask.append(seq_mask)
 
  inputs = torch.tensor(input_ids)
  masks = torch.tensor(attention_mask)
 
  return inputs, masks

   


def test_sentences(sentences):
  model.eval()
  inputs, masks = convert_input_data(sentences)
  b_input_ids = inputs.to(device)
  b_input_masks = masks.to(device)
  with torch.no_grad():
    outputs = model(b_input_ids,
                    token_type_ids = None,
                    attention_mask = b_input_masks)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
      
    return logits
 

In [23]:
for i in range(20):
    t = [df.iloc[i].review]
    print(t)
    print(np.argmax(test_sentences(t)))
    print(classes_oh[i])

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f